# Use of the static grid equivalents
This tutorial gives a quick overview of the grid equivalents function. This script gives an example to reduce a grid area using the IEEE case9 grid.

## Determination of grid areas

The user can determine the grid area of interest (internal area), the grid area for the reduction (external area), and the boundary buses between them. The figure below schows an example of the grid with identified grid areas:

- internal area (buses): [0, 3, 4, 8]
- boundary buses: [4, 8] (boundary buses belong to the internal area)
- external area (buses): [1, 2, 5, 6, 7] 

<img src="pics/grid areas.png" alt="ALT">

In order to identify the user-defined area, the boundary buses and one of the internal buses must be provided as inputs.


## Required inputs

The most important inputs for grid equivalents are:
- net: pandapower grid including power flow results
- eq_type: the method used to determine the equivalent grid, e.g., "rei", "ward", "xward"
- boundary_buses: indices of boundary buses (as a list) that divide the original grid into the internal areas and the external areas. Based on the figure above, boundary_buses = [4, 8] 
- internal_buses: bus indices (as a list), which are within the internal area. Based on the figure above, internal_buses = [0]. Just one of the internal buses is enough, the function will find and consider the remaining internal buses during the equivalent calculation. If 'internal_buses' is an empty list or None, the complete grid is treated as an external area.

## Example: REI-equivalent
In the following, the reduction of the external grid (buses [1, 2, 5, 6, 7] of the grid case 9) in the figure above is shown.
First, the necessary libraries need to be imported.

In [1]:
import pandapower as pp
import pandapower.networks
import pandapower.grid_equivalents

Subsequently, the IEEE case 9 grid model is created, and its power flow is calculated.

In [2]:
net = pp.networks.case9()
net.sn_mva = 1.0
pp.runpp(net)
net

This pandapower network includes the following parameter tables:
   - bus (9 elements)
   - load (3 elements)
   - gen (2 elements)
   - ext_grid (1 element)
   - line (9 elements)
   - poly_cost (3 elements)
   - bus_geodata (9 elements)
 and the following results tables:
   - res_bus (9 elements)
   - res_line (9 elements)
   - res_ext_grid (1 element)
   - res_load (3 elements)
   - res_gen (2 elements)

In [3]:
# equivalent type
eq_type = "rei"

# boundary buses
boundary_buses = [4, 8]

# internal buses 
internal_buses = [0]

Next, the equivalent function is called, and the grid reduction is executed.

In [4]:
net_eq = pp.grid_equivalents.get_equivalent(net, eq_type, boundary_buses, internal_buses)

hp.pandapower.grid_equivalents.get_equivalent - INFO: rei equivalent calculation started
hp.pandapower.toolbox - INFO: dropped 2 gen elements
hp.pandapower.toolbox - INFO: dropped 0 switches
hp.pandapower.toolbox - INFO: dropped 1 load elements
hp.pandapower.toolbox - INFO: dropped 5 lines with 0 line switches
hp.pandapower.toolbox - INFO: dropped 1 lines with 0 line switches
hp.pandapower.toolbox - INFO: dropped 0 switches
hp.pandapower.toolbox - INFO: dropped 2 lines with 0 line switches
hp.pandapower.toolbox - INFO: dropped 1 ext_grid elements
hp.pandapower.toolbox - INFO: dropped 1 lines with 0 line switches
hp.pandapower.toolbox - INFO: dropped 3 impedance elements
hp.pandapower.toolbox - INFO: dropped 0 switches
hp.pandapower.toolbox - INFO: dropped 5 lines with 0 line switches
hp.pandapower.toolbox - INFO: dropped 3 impedance elements
hp.pandapower.toolbox - INFO: dropped 1 lines with 0 line switches
hp.pandapower.toolbox - INFO: dropped 0 lines with 0 line switches
hp.pandapowe

Now, the grid equivalent is obtained: "net_eq" is the REI-equivalent grid model. We can compare the power flow results between the original grid "net" and the reduced grid "net_eq".  

In [5]:
print("--- power flow (original grid) ---")
display(net.res_bus)
print("--- power flow (reduced grid) ---")
display(net_eq.res_bus)

--- power flow (original grid) ---


,vm_pu,va_degree,p_mw,q_mvar
0,1.000000,0.000000,-71.954702,-24.068958
1,1.000000,9.668741,-163.000000,-14.460120
2,1.000000,4.771073,-85.000000,3.649026
3,0.987007,-2.406644,0.000000,0.000000
4,0.975472,-4.017264,90.000000,30.000000
5,1.003375,1.925602,0.000000,0.000000
6,0.985645,0.621545,100.000000,35.000000
7,0.996185,3.799120,0.000000,0.000000
8,0.957621,-4.349934,125.000000,50.000000


--- power flow (reduced grid) ---


,vm_pu,va_degree,p_mw,q_mvar
0,1.000000,0.000000,-71.954702,-24.068958
3,0.987007,-2.406644,0.000000,0.000000
4,0.975472,-4.017264,89.283896,8.224645
8,0.957621,-4.349934,124.581896,32.419383
9,0.985645,0.621545,99.317421,0.635364
10,1.000000,9.668741,-162.192474,-24.665476
11,1.000000,4.771073,-83.866102,-11.311074


It can be seen that the power flow results (*vm_pu*, *va_degree*) of the internal buses [0, 3, 4, 8] in both grids are the same , i.e., the equivalent calculation is successful. The *p_mw* and *q_mvar* values at the boundary buses [4, 8] are different due to the equivalent devices. During the grid reduction, additional equivalent devices (e.g., additional buses [9, 10, 11], impedance, shunts, etc.) are created, representing the interaction between the internal area and the external area, and maintaining the grid states in the internal area. We can see the additional shunts in the grid equivalent:

In [6]:
print("--- shunts (original grid) ---")
display(net.shunt)
print("--- shunts (reduced grid) ---")
display(net_eq.shunt)

--- shunts (original grid) ---


,bus,name,q_mvar,p_mw,vn_kv,step,max_step,in_service


--- shunts (reduced grid) ---


,bus,name,q_mvar,p_mw,vn_kv,step,max_step,in_service
0,4,eq_shunt,-22.884186,-0.752570,345.0,1,1,True
1,8,eq_shunt,-19.171087,-0.455928,345.0,1,1,True
2,9,eq_shunt,-35.372912,-0.702606,345.0,1,1,True
3,10,eq_shunt,-10.205356,0.807526,345.0,1,1,True
4,11,eq_shunt,-14.960100,1.133898,345.0,1,1,True


## Example: (X)Ward-equivalent

In the following, we demonstrate an example for the (x)ward-equivalent. We change the equivalent type and repeat the equivalent calculation.

In [7]:
# equivalent type
eq_type = "ward"   # for xward-equivalent: eq_type = "xward"

# run equivalent calculation
net_eq_ward = pp.grid_equivalents.get_equivalent(net, eq_type, boundary_buses, 
                                                 internal_buses)

hp.pandapower.grid_equivalents.get_equivalent - INFO: ward equivalent calculation started
hp.pandapower.toolbox - INFO: dropped 2 gen elements
hp.pandapower.toolbox - INFO: dropped 0 switches
hp.pandapower.toolbox - INFO: dropped 1 load elements
hp.pandapower.toolbox - INFO: dropped 5 lines with 0 line switches
hp.pandapower.toolbox - INFO: dropped 1 lines with 0 line switches
hp.pandapower.toolbox - INFO: dropped 0 switches
hp.pandapower.toolbox - INFO: dropped 2 lines with 0 line switches
hp.pandapower.toolbox - INFO: dropped 1 ext_grid elements
hp.pandapower.toolbox - INFO: dropped 1 lines with 0 line switches
hp.pandapower.toolbox - INFO: dropped 2 gen elements
hp.pandapower.toolbox - INFO: dropped 0 switches
hp.pandapower.toolbox - INFO: dropped 1 load elements
hp.pandapower.toolbox - INFO: dropped 5 lines with 0 line switches
hp.pandapower.toolbox - INFO: dropped 1 lines with 0 line switches
hp.pandapower.toolbox - INFO: dropped 0 lines with 0 line switches
hp.pandapower.toolbox 

Now, the ward equivalent is obtained. We can verify the power flow results.

In [8]:
print("--- power flow (original grid) ---")
display(net.res_bus)
print("--- power flow (reduced grid) ---")
display(net_eq_ward.res_bus)

--- power flow (original grid) ---


,vm_pu,va_degree,p_mw,q_mvar
0,1.000000,0.000000,-71.954702,-24.068958
1,1.000000,9.668741,-163.000000,-14.460120
2,1.000000,4.771073,-85.000000,3.649026
3,0.987007,-2.406644,0.000000,0.000000
4,0.975472,-4.017264,90.000000,30.000000
5,1.003375,1.925602,0.000000,0.000000
6,0.985645,0.621545,100.000000,35.000000
7,0.996185,3.799120,0.000000,0.000000
8,0.957621,-4.349934,125.000000,50.000000


--- power flow (reduced grid) ---


,vm_pu,va_degree,p_mw,q_mvar
0,1.000000,0.000000,-71.954702,-24.068958
3,0.987007,-2.406644,0.000000,0.000000
4,0.975472,-4.017264,28.820879,10.232062
8,0.957621,-4.349934,42.681861,39.100723


The power flow results (*vm_pu*, *va_degree*) for the internal buses [0, 3, 4, 8] in both grids are the same. The external area is represented by the addtional *ward* elements in the grid model "net_eq_ward", attached at the boundary buses [4, 8]: 

In [9]:
print("--- ward (original grid) ---")
display(net.ward)
print("--- ward (reduced grid) ---")
display(net_eq_ward.ward)

--- ward (original grid) ---


,name,bus,ps_mw,qs_mvar,qz_mvar,pz_mw,in_service


--- ward (reduced grid) ---


,name,bus,ps_mw,qs_mvar,qz_mvar,pz_mw,in_service
0,grid_equivalent,4,-61.254284,31.921715,-54.321760,0.07899,True
1,grid_equivalent,8,-83.137118,36.970310,-52.200219,0.89307,True
